# 

# Detecção de conflitos em projetos BIM usando grafos e ontologias

## Introdução
Este projeto tem como objetivo detectar conflitos em projetos BIM (Building Information Modeling) utilizando grafos e ontologias. A detecção de conflitos é uma etapa crucial no processo de planejamento e execução de projetos de construção, pois ajuda a identificar problemas potenciais em projetos antes que eles se tornem caros ou difíceis de resolver.

## Tecnologias Utilizadas
- **Python**: Linguagem de programação principal para o desenvolvimento do projeto.
- **PyShacl**: Biblioteca para manipulação de grafos.
- **RDFLib**: Biblioteca para trabalhar com grafos RDF (Resource Description Framework).
- **Ontologia**: Definição de classes, propriedades e relações entre os elementos do projeto BIM.
- **Neo4j**: Banco de dados orientado a grafos para armazenar e consultar os dados do projeto.

### Importando as bibliotecas necessárias

In [1]:
import ifcopenshell
from rdflib import Graph, Namespace, URIRef, Literal, RDF, RDFS, OWL
import re
from rdflib.namespace import SH
from pyshacl import validate
from py2neo import Graph as Neo4jGraph, Node, Relationship

### Definindo Variáveis e constantes

In [2]:
IFC_FILE = "../data/Building-Architecture.ifc"
model = ifcopenshell.open(IFC_FILE)
produtos = model.by_type("IfcProduct")

NEO4J_URI = "bolt://localhost:7687"
NEO4J_USER = "neo4j"
NEO4J_PASSWORD = "ifc123456"
NEO4J_DATABASE = "ifctest"

IFC_NS = Namespace("http://example.org/ifc/")
PROP_NS = Namespace("http://example.org/ifc/property#")
SHACL_NS = Namespace("http://www.w3.org/ns/shacl#")

### Inicializando o grafo

In [3]:
rdf_graph = Graph()
rdf_graph.bind("ifc", IFC_NS)
rdf_graph.bind("prop", PROP_NS)

### Carregando os dados do modelo IFC no neo4j

os dados do modelo IFC serão carregados no banco de dados Neo4j, onde cada elemento do modelo será representado como um nó no grafo. As relações entre os elementos serão representadas como arestas.

In [4]:
neo4j_graph = Neo4jGraph(
    uri=NEO4J_URI,
    auth=(NEO4J_USER, NEO4J_PASSWORD),
)

# Limpar o grafo Neo4j antes de inserir novos dados
neo4j_graph.delete_all()

# Função auxiliar para converter GlobalId para URI
def globalid_to_uri(global_id):
    return URIRef(IFC_NS[global_id])

# Função para adicionar entidades ao grafo Neo4j
def add_entity_to_neo4j(entity):
    uri = globalid_to_uri(entity.GlobalId)
    node = Node("IfcProduct", global_id=entity.GlobalId, name=entity.Name or "")
    neo4j_graph.create(node)
    
    for key, value in entity.get_info(recursive=False).items():
        if key not in ['GlobalId', 'OwnerHistory', 'Name', 'type']:
            if isinstance(value, ifcopenshell.entity_instance):
                # Only create relationship if the referenced entity has a GlobalId
                if hasattr(value, "GlobalId") and value.GlobalId:
                    rel = Relationship(node, key, Node("IfcProduct", global_id=value.GlobalId))
                    neo4j_graph.create(rel)
            elif isinstance(value, (list, tuple)):
                for item in value:
                    if isinstance(item, ifcopenshell.entity_instance):
                        if hasattr(item, "GlobalId") and item.GlobalId:
                            rel = Relationship(node, key, Node("IfcProduct", global_id=item.GlobalId))
                            neo4j_graph.create(rel)
                        
for produto in produtos:
    if not produto.GlobalId:
        continue
    add_entity_to_neo4j(produto)

### Populando entidades e propriedades Basicas no grafo do RDF
Nesta etapa, serão criadas as entidades e propriedades básicas no grafo RDF, representando os elementos do projeto BIM. Isso inclui a definição de classes para diferentes tipos de elementos, como paredes, portas, janelas, etc., e suas propriedades associadas.

In [5]:
def populate_rdf_graph():
    for element in produtos:
        if not element.GlobalId:
            continue
            
        uri = URIRef(IFC_NS[element.GlobalId])
        
        rdf_graph.add((uri, RDF.type, URIRef(IFC_NS[element.is_a()])))
        
        if element.Name:
            rdf_graph.add((uri, IFC_NS["name"], Literal(element.Name)))
        
        # Adicionar propriedades de relacionamento
        info = element.get_info(recursive=False)
        for key, value in info.items():
            if key in ['GlobalId', 'OwnerHistory', 'Name', 'type']:
                continue
                
            if isinstance(value, ifcopenshell.entity_instance):
                if value.is_a("IfcProduct"):
                    ref_uri = URIRef(IFC_NS[value.GlobalId])
                    rdf_graph.add((uri, PROP_NS[key], ref_uri))
                    
            elif isinstance(value, (list, tuple)):
                for item in value:
                    if isinstance(item, ifcopenshell.entity_instance) and item.is_a("IfcProduct"):
                        ref_uri = URIRef(IFC_NS[item.GlobalId])
                        rdf_graph.add((uri, PROP_NS[key], ref_uri))

populate_rdf_graph()

## Introdução ao SHACL
SHACL (Shapes Constraint Language) é uma linguagem para descrever restrições de **forma** em grafos RDF. Ela permite definir regras que os dados devem seguir, facilitando a validação e a detecção de conflitos.

### Definindo as formas SHACL

- Regra 1: Paredes devem estar contidas em estruturas espaciais
- Regra 2: Portas devem ser instaladas em paredes
- Regra 3: Pisos devem ter relações com edificações
- Regra 4: Relações espaciais devem ter tipos válidos

In [6]:
shacl_shapes = f"""
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix sh: <{SH}> .
@prefix ifc: <{IFC_NS}> .
@prefix prop: <{PROP_NS}> .

# Regra 1: Paredes devem estar contidas em estruturas espaciais
ifc:WallContainmentRule
    a sh:NodeShape ;
    sh:targetClass ifc:IfcWall ;
    sh:property [
        sh:path prop:ContainedInStructure ;
        sh:class ifc:IfcSpatialStructureElement ;
        sh:minCount 1 ;
        sh:maxCount 1 ;
        sh:message "Paredes devem estar contidas em exatamente uma estrutura espacial (andar/edifício)" ;
    ] .

# Regra 2: Portas devem ser instaladas em paredes
ifc:DoorInstallationRule
    a sh:NodeShape ;
    sh:targetClass ifc:IfcDoor ;
    sh:property [
        sh:path prop:FillsVoids ;
        sh:class ifc:IfcWall ;
        sh:minCount 1 ;
        sh:message "Portas devem ser instaladas em paredes" ;
    ] .

# Regra 3: Pisos devem ter relações com edificações
ifc:SlabContainmentRule
    a sh:NodeShape ;
    sh:targetClass ifc:IfcSlab ;
    sh:property [
        sh:path prop:ContainedInStructure ;
        sh:class ifc:IfcBuilding ;
        sh:minCount 1 ;
        sh:message "Pisos devem estar contidos em edificações" ;
    ] .

# Regra 4: Relações espaciais devem ter tipos válidos
ifc:ValidRelationRule
    a sh:PropertyShape ;
    sh:path prop:RelatingSpace ;
    sh:class ifc:IfcSpace ;
    sh:message "Elementos só podem relacionar-se com espaços válidos" .

# TODO: Adicionar regras adicionais conforme necessário
# TODO: REgra 5: Elementos de construção devem ter propriedades definidas
# TODO: Regra 6: Janelas devem estar instaladas em paredes
# TODO: Regra 7: Tetos devem estar contidos em edificações
"""

### Validando o Grafo com as regras SHACL
A validação do grafo com as regras SHACL é realizada para verificar se os dados do projeto BIM atendem às restrições definidas. Isso ajuda a identificar conflitos e inconsistências nos dados.

In [ ]:
shacl_graph = Graph().parse(data=shacl_shapes, format="turtle")
conforms, results_graph, results_text = validate(
    rdf_graph,
    shacl_graph=shacl_graph,
    ont_graph=None,
    inference='rdfs',
    abort_on_first=False,
    meta_shacl=False,
    advanced=True,
    debug=False
)

In [ ]:
print("\n" + "="*50)
print("Resultado da Validação de Conformidade")
print("="*50)

if conforms:
    print("Nenhum conflito detectado: O modelo está em conformidade com a ontologia")
else:
    print(f"Foram encontrados {len(results_graph)} conflitos ontológicos:")
    
    conflicts = []
    for s in results_graph.subjects(SH.resultSeverity, SH.Violation):
        conflict = {
            "element": results_graph.value(s, SH.focusNode),
            "message": results_graph.value(s, SH.resultMessage),
            "path": results_graph.value(s, SH.resultPath),
            "severity": "Violação"
        }
        conflicts.append(conflict)
        
    for i, conflict in enumerate(conflicts, 1):
        print(f"\nConflito #{i}:")
        print(f"Elemento: {conflict['element']}")
        print(f"Mensagem: {conflict['message']}")
        print(f"Propriedade: {conflict['path']}")
        print(f"Severidade: {conflict['severity']}")

# rdf_graph.serialize("ifc_graph.ttl", format="turtle")
# with open("validation_report.txt", "w") as f:
#     f.write(results_text)

print("\nValidação concluída. Relatório salvo em 'validation_report.txt'")


Resultado da Validação de Conformidade
Foram encontrados 67 conflitos ontológicos:

Conflito #1:
Elemento: http://example.org/ifc/3zR0BOEcLADRKln4HYporH
Mensagem: Pisos devem estar contidos em edificações
Propriedade: http://example.org/ifc/property#ContainedInStructure
Severidade: Violação

Conflito #2:
Elemento: http://example.org/ifc/12UVOn4wvAJPMUExKdZLb8
Mensagem: Pisos devem estar contidos em edificações
Propriedade: http://example.org/ifc/property#ContainedInStructure
Severidade: Violação

Conflito #3:
Elemento: http://example.org/ifc/0ZTBBPo6f6bxqV2K7Oelrq
Mensagem: Pisos devem estar contidos em edificações
Propriedade: http://example.org/ifc/property#ContainedInStructure
Severidade: Violação

Conflito #4:
Elemento: http://example.org/ifc/0OfZwWc8j9QP5uX8xPTxDH
Mensagem: Paredes devem estar contidas em exatamente uma estrutura espacial (andar/edifício)
Propriedade: http://example.org/ifc/property#ContainedInStructure
Severidade: Violação

Conflito #5:
Elemento: http://example.